![](https://salin.fb.utfpr.edu.br/img/semana.png)

# Oficina: Introdução à Programação Quântica com Ket

A computação quântica é uma tecnologia emergente com o potencial de revolucionar diversos setores industriais até o final desta década, quando os computadores quânticos prometem ser mais eficientes do que supercomputadores. Embora seja necessário um computador quântico para aproveitar plenamente suas vantagens, já podemos começar a desenvolver aplicações quânticas em computadores tradicionais. Nesta oficina, apresentaremos uma introdução à programação de computadores quânticos usando a plataforma Ket, explorando os postulados da mecânica quântica e experimentando na prática como escrever código quântico.

In [ ]:
!pip install -U pip -q --root-user-action=ignore
!pip install -U ket-lang[visualization]==0.7.1 -q --root-user-action=ignore

from math import pi, sqrt, radians

from IPython.display import display
from ket import *
from ket import __version__

print(f"Ket version {__version__}")

## Notação de Dirac

A Mecânica Quântica adota uma notação distinta daquela à qual estamos habituados na Álgebra Linear. Embora essa notação possa inicialmente parecer estranha, seu propósito é simplificar os cálculos na Mecânica Quântica.

- Numero Complexo
    - $z = a + ib, \ z \in \mathbb{C}, \ a, b \in \mathbb{R}, \ i = \sqrt{-1}$
    - $e^{i\theta} = \cos{\theta}+i\sin{\theta}, \ \theta \in \mathbb{R}$
  
    ![](https://upload.wikimedia.org/wikipedia/commons/thumb/7/71/Euler%27s_formula.svg/360px-Euler%27s_formula.svg.png)
- Conjugado
  - $z^\dagger = \bar{z} = (a + bi)^\dagger = a-bi$
- Conjugado Transposto *ou* Hermitiano de Matriz
  - $A^\dagger \Rightarrow (A^\dagger)_{ji} = \overline{ A_{ij} }$
- Vetor Coluna *Ket*
  - $\left|\psi\right> = \begin{bmatrix}a\\b\\c\\\vdots\end{bmatrix}$  
- Vetor Linha *Bra*
  - $\left<\psi\right| = \left|\psi\right>^\dagger = \begin{bmatrix}a^\dagger & b^\dagger & c^\dagger & \dots\end{bmatrix}$
- Produto interno
  - $\left<\varphi|\psi\right> = \begin{bmatrix}a & b & c & \dots\end{bmatrix}\begin{bmatrix}x\\y\\z\\\vdots\end{bmatrix} = ax+by+cz+\dots$
- Norma (tamanho do vetor)
  - $\|\left|\psi\right>\| = \sqrt{\left<\psi|\psi\right>} = \sqrt{|a|^2+|b|^2+|c|^2+\dots}$
- Produto externo
  - ${\left|\varphi\right>}{\left<\psi\right|} = \begin{bmatrix}a \\ b \\ c \\ \vdots\end{bmatrix}\begin{bmatrix}x & y & z  & \dots\end{bmatrix}= \begin{bmatrix}ax & ay & az & \dots \\ bx & by & bz & \dots \\ cx & cy & cz & \dots \\ \vdots & \vdots & \vdots & \ddots\end{bmatrix}$

## Postulado 1 - Espaço do Sistema

> Associado a qualquer sistema físico isolado, existe um espaço de vetores complexos com produto interno (ou seja, um espaço de Hilbert), conhecido como o espaço de estados do sistema. O sistema é completamente descrito pelo seu vetor de estado, que é um vetor unitário no espaço de estados do sistema.

Um exemplo fundamental desse conceito é o qubit, ou bit quântico. Um qubit é representado por um vetor unitário em $\mathbb{C}^2$, geralmente expresso como uma combinação linear da base canônica, também conhecida como base computacional. Assim, qualquer qubit pode ser descrito como:

$$
\left| \psi \right\rangle = \alpha\left| 0 \right\rangle + \beta\left| 1 \right\rangle = \begin{bmatrix} \alpha \\ \beta \end{bmatrix}
$$

Onde:

$$
\left| 0 \right\rangle = \begin{bmatrix} 1 \\ 0 \end{bmatrix}, \quad \left| 1 \right\rangle = \begin{bmatrix} 0 \\ 1 \end{bmatrix}
$$

E $\alpha$ e $\beta$ são coeficientes complexos chamados de amplitude de probabilidade que satisfazem a condição de normalização:

$$
|\alpha|^2 + |\beta|^2 = 1
$$

### Alocando Qubit no Ket

In [ ]:
process = Process()

qubit = process.alloc()

print("Estado inicial do qubit:")
dump(qubit).show(mode="latex")

## Postulado 2 - Evolução

> A evolução de um sistema quântico fechado é descrita por uma transformação unitária. Isso significa que o estado $\left| \psi_1 \right\rangle$ do sistema no tempo $t_1$ está relacionado ao estado $\left| \psi_2 \right\rangle$ do sistema no tempo $t_2$ por um operador unitário $U$, que depende apenas dos tempos $t_1$ e $t_2$, conforme a equação:
> 
> $$U\left| \psi_1 \right\rangle = \left| \psi_2 \right\rangle$$

Na computação quântica, chamamos essas operações unitárias de portas lógicas quânticas. Exemplos de portas incluem:

- Porta X:
$$X = \begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix}$$
$$X\left|0\right\rangle = \left|1\right\rangle, \quad X\left|1\right\rangle = \left|0\right\rangle$$

- Porta Y:
$$Y = \begin{bmatrix} 0 & -i \\ i & 0 \end{bmatrix}$$
$$Y\left|0\right\rangle = i\left|1\right\rangle, \quad Y\left|1\right\rangle = -i\left|0\right\rangle$$

- Porta Z:
$$Z = \begin{bmatrix} 1 & 0 \\ 0 & -1 \end{bmatrix}$$
$$Z\left|0\right\rangle = \left|0\right\rangle, \quad Z\left|1\right\rangle = -\left|1\right\rangle$$

- Porta Hadamard:
$$H = \frac{1}{\sqrt{2}}\begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix}$$
$$H\left|0\right\rangle = \frac{\left|0\right\rangle + \left|1\right\rangle}{\sqrt{2}} = \left|+\right\rangle, \quad H\left|1\right\rangle = \frac{\left|0\right\rangle - \left|1\right\rangle}{\sqrt{2}} = \left|-\right\rangle$$
$$H\left|+\right\rangle = \left|0\right\rangle, \quad H\left|-\right\rangle = \left|1\right\rangle$$

- Porta CNOT (Controlled-NOT):
$$\text{CNOT} = \begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0 \end{bmatrix}$$
$$\text{CNOT}\left|00\right\rangle = \left|00\right\rangle, \quad \text{CNOT}\left|01\right\rangle = \left|01\right\rangle, \quad \text{CNOT}\left|10\right\rangle = \left|11\right\rangle, \quad \text{CNOT}\left|11\right\rangle = \left|10\right\rangle$$

O Ket implementa diversas portas lógicas quânticas que permitem implementar qualquer computação quântica. Consulte a documentação do Ket para ver as portas disponíveis: [Documentação do Ket](https://quantumket.org/ket/api/ket.gates.html).

### Aplicação de Porta Quântica no Ket

In [ ]:
process = Process()

qubit = process.alloc()

X(qubit)
H(qubit)


dump(qubit).show(mode="latex")

### Esfera de Bloch

A Esfera de Bloch é uma representação geométrica tridimensional de um bit quântico, que oferece uma maneira intuitiva de visualizar e entender seu estado. Nesta esfera, cada ponto na superfície representa um estado possível do qubit. Por exemplo, os estados básicos $|0\rangle$ e $|1\rangle$ são representados pelos polos norte e sul da esfera, respectivamente. Além disso, estados superpostos são representados por pontos em outras posições da superfície da esfera.

Uma característica importante da Esfera de Bloch é que os estados ortogonais são representados por pontos diametralmente opostos na esfera. Além de fornecer uma representação visual dos estados de um qubit, a Esfera de Bloch também é útil para entender operações quânticas. Por exemplo, rotações ao redor dos eixos da esfera representam operações de porta quântica, como a aplicação de portas de Pauli e a operação de Hadamard.

In [ ]:
process = Process()

qubit = process.alloc()

RX(radians(-2.5), qubit)

dump(qubit).sphere().show()

## Postulado 3 - Medida

> Medições quânticas são descritas por uma coleção $\{M_m\}$ de operadores de medição. Esses são operadores que atuam no espaço de estados do sistema que está sendo medido. O índice $m$ se refere aos resultados de medição que podem ocorrer no experimento. Se o estado do sistema quântico é $\left| \psi \right\rangle$ imediatamente antes da medição, então a probabilidade de que o resultado $m$ ocorra é dada por:
> 
> $$ p(m) = \left\langle \psi \right|M_m^\dagger M_m\left| \psi \right\rangle, $$
> 
> E o estado do sistema após a medição é:
> 
> $$ \frac{M_m\left| \psi \right\rangle}{\sqrt{\left\langle \psi \right|M_m^\dagger M_m\left| \psi \right\rangle}}. $$
> 
> Os operadores de medição satisfazem a equação de completude:
> 
> $$ \sum_m M_m^\dagger M_m = I. $$

O Ket oferece medidas na base computacional, uma funcionalidade essencial comumente empregada em computadores quânticos. Na base computacional, a coleção de medidas é representada por:

$$
\left\{ M_0 = {\left| 0 \right\rangle }{ \left\langle 0 \right| } = \begin{bmatrix} 1 & 0 \\ 0 & 0 \end{bmatrix}, \quad M_1 = {\left| 1 \right\rangle}{ \left\langle 1 \right| } = \begin{bmatrix} 0 & 0 \\ 0 & 1 \end{bmatrix}  \right\}
$$

Probabilidade de medir 0 em $\left| \psi \right\rangle = \alpha\left| 0 \right\rangle+\beta\left| 1 \right\rangle$:

$$
\begin{aligned}
    p(0) & = \left(\alpha^\dagger\left< 0 \right|+\beta^\dagger\left< 1 \right|\right){\left| 0 \right>}{\left< 0 | 0 \right>}{\left<0\right|} \left(\alpha\left| 0 \right>+\beta\left| 1 \right>\right) \\
    & = \left(\alpha^\dagger\left<0|0\right>+\beta^\dagger\left<1|0\right>\right)\left<0|0\right>\left(\alpha\left<0|0\right>+\beta\left<0|1\right>\right) \\
    & = \alpha^\dagger\alpha = |\alpha|^2
\end{aligned}
$$

Estado após medir 0:

$$
\begin{aligned}
    \frac{M_0\left| \psi \right\rangle}{\sqrt{\left\langle \psi \right|M_0^\dagger M_0\left| \psi \right\rangle}}
    & = \frac{{\left|0\right\rangle}{\left\langle0\right|}\left(\alpha\left| 0 \right\rangle+\beta\left| 1 \right\rangle\right)}{\sqrt{|\alpha|^2}} \\
    & = \frac{\left|0\right\rangle\left(\alpha\left\langle0|0\right\rangle+\beta\left\langle0|1\right\rangle\right)} {|\alpha|} = \frac{\alpha\left|0\right\rangle} {|\alpha|}
\end{aligned}
$$

### Medidas no Ket

In [ ]:
process = Process()

qubit = process.alloc()
T(H(qubit))

print("Qubit antes de medir:")
display(dump(qubit).show(mode="latex"))
dump(qubit).histogram().show()

m = measure(qubit)

print("Resultado da medida:", m.value)
print()

print("Qubit depois de medir:")
display(dump(qubit).show(mode="latex"))
dump(qubit).histogram().show()

In [ ]:
process = Process()

qubit = process.alloc()
H(qubit)

m = sample(qubit)

print("Resultado da amostragem:", m.value)

## Postulado 4 - Sistema Composto

> O espaço de um sistema físico composto é o produto tensorial dos subespaços. Além disso, se temos sistemas numerados de $1$ a $n$, e o sistema número $i$ é preparado no estado $\left| \psi_i \right>$, então o estado conjunto do sistema total é $\left| \psi_1 \right>\otimes\left| \psi_2 \right>\otimes\cdots\otimes\left| \psi_n \right>$.

Produto tensorial:
$$
A \otimes B = \begin{bmatrix}
            A_{11}B & A_{12}B & \cdots  & A_{1n}B  \\
            A_{11}B & A_{12}B & \cdots  & A_{1n}B  \\
            A_{21}B & A_{22}B & \cdots  & A_{2n}B  \\
            \vdots  & \vdots  & \ddots & \vdots    \\
            A_{m1}B & A_{m2}B & \cdots  & A_{mn}B  
        \end{bmatrix}
$$

## Preparação do estado de Bell:

$$
\text{CNOT}(H_0\left| 00 \right>)  = \frac{1}{\sqrt{2}}\left(\left| 00 \right>+\left| 11 \right>\right)
$$

$$
\begin{aligned}
    \left(\frac{1}{\sqrt{2}}\begin{bmatrix}
        1 & 1 \\ 1 & -1
    \end{bmatrix}\otimes\begin{bmatrix}
        1 & 0 \\ 0 & 1
    \end{bmatrix}\right)\left(\begin{bmatrix}
        1 \\ 0 
    \end{bmatrix}\otimes\begin{bmatrix}
        1 \\ 0
    \end{bmatrix}\right) & = \frac{1}{\sqrt{2}} \begin{bmatrix}
        1 & 0 & 1 & 0 \\
        0 & 1 & 0 & 1 \\
        1 & 0 & -1 & 0 \\
        0 & 1 & 0 & -1 
    \end{bmatrix}\begin{bmatrix}
        1 \\ 0 \\ 0 \\ 0 
    \end{bmatrix} = \frac{1}{\sqrt{2}}\begin{bmatrix}
        1 \\ 0 \\ 1 \\ 0 
    \end{bmatrix} \\
    \begin{bmatrix}
        1 & 0 & 0 & 0 \\
        0 & 1 & 0 & 0 \\
        0 & 0 & 0 & 1 \\
        0 & 0 & 1 & 0
    \end{bmatrix} \frac{1}{\sqrt{2}}\begin{bmatrix}
        1 \\ 0 \\ 1 \\ 0 
    \end{bmatrix} & = \frac{1}{\sqrt{2}}\begin{bmatrix}
        1 \\ 0 \\ 0 \\ 1 
    \end{bmatrix}
\end{aligned}
$$


In [ ]:
process = Process()

a, b = process.alloc(2)

print("Estado Inicial:")
display(dump(a + b).show(mode="latex"))

H(a)
print("Aplicando Hadamard:")
display(dump(a + b).show(mode="latex"))

CNOT(a, b)
print("Aplicando CNOT:")
display(dump(a + b).show(mode="latex"))

m = measure(a + b)
print(f"Medir a: (resultado = {m.value})")
display(dump(a + b).show(mode="latex"))

#### Implementações alternativas

In [ ]:
process = Process()

a, b = process.alloc(2)

H(a)
CNOT(a, b)

display(dump(a + b).show(mode="latex"))

In [ ]:
process = Process()

a, b = process.alloc(2)

CNOT(H(a), b)

display(dump(a + b).show(mode="latex"))

In [ ]:
process = Process()

a, b = process.alloc(2)

bell = cat(kron(H, I), CNOT)
bell(a, b)

display(dump(a + b).show(mode="latex"))

In [ ]:
process = Process()

q = process.alloc(2)

cat(kron(H, I), CNOT)(*q)


display(dump(q).show(mode="latex"))

In [ ]:
process = Process()

a, b = process.alloc(2)

H(a)
ctrl(a, X)(b)

display(dump(a + b).show(mode="latex"))

-------

## Teleporte Quântico

O teleporte quântico é um protocolo que permite transferir a informação de um qubit para outro usando um canal quântico (emaranhamento) e dois canais clássicos.

<img src="https://upload.wikimedia.org/wikipedia/commons/d/dc/Quantum_teleportation_circuit.svg" alt="teleporte" width="500"/>

O estado inicialmente compartilhado entre Alice e Bob é conhecido como estado de Bell, representado por:

$$\left| \Phi^+ \right\rangle = \frac{\left|00\right\rangle + \left|11\right\rangle}{\sqrt{2}}$$

In [ ]:
def entangle(a, b):
    return CNOT(H(a), b)


def teleport(quantum_message, entangled_qubit):
    adj(entangle)(quantum_message, entangled_qubit)
    return measure(entangled_qubit), measure(quantum_message)


def decode(classical_message, qubit):
    if classical_message[0].value == 1:
        X(qubit)
    if classical_message[1].value == 1:
        Z(qubit)


p = Process()

alice_qubit, bob_qubit = entangle(*p.alloc(2))

print("Alice e Bob: Qubits entrelaçados")
display(dump(alice_qubit + bob_qubit).show(mode="latex"))

alice_message = PHASE(pi / 4, H(p.alloc()))

print("Alice: Mensagem")
display(dump(alice_message).show(mode="latex"))

classical_message = teleport(
    quantum_message=alice_message,
    entangled_qubit=alice_qubit,
)

decode(classical_message, bob_qubit)

print("Bob: Estado final")
display(dump(bob_qubit).show(mode="latex"))

-----

### Algoritmo Quântico de Busca (Algoritmo de Grover)

O algoritmo de Grover é uma técnica quântica para buscar uma entrada específica em uma lista não ordenada de $N = 2^n$ itens em tempo $O(\sqrt{N})$, em oposição à busca clássica com tempo linear $O(N)$. 

![](https://upload.wikimedia.org/wikipedia/commons/thumb/b/b9/Grover%27s_algorithm_circuit.svg/1000px-Grover%27s_algorithm_circuit.svg.png)

A operação $U_w$ em Grover é definida como:

$$
U_w |x\rangle = \begin{cases} |x\rangle & \text{se } x \neq w \\ -|x\rangle & \text{se } x = w \end{cases}
$$

Essencialmente, $U_w$ inverte o sinal do estado $|w\rangle$ enquanto mantém os outros estados inalterados. Isso é fundamental para a amplificação da amplitude do estado desejado durante a execução do algoritmo de Grover.

In [ ]:
def grover(size: int, oracle) -> int:
    p = Process(simulator="dense", num_qubits=size)

    s = H(p.alloc(size))
    steps = int((pi / 4) * sqrt(2**size))
    for _ in range(steps):
        oracle(s)
        with around(cat(H, X), s):
            ctrl(s[:-1], Z)(s[-1])

    return measure(s).value


def randint(n_bits: int) -> int:
    process = Process(simulator="dense", num_qubits=n_bits)
    qubits = process.alloc(n_bits)
    H(qubits)
    return measure(qubits).value


n_qubits = 12
looking_for = randint(n_qubits)

print("Procurando por", looking_for, "usando", n_qubits, "qubits.")

print(
    "Dense Simulation: resultado medido",
    grover(n_qubits, lib.phase_oracle(looking_for)),
)

------- 

# Veja mais

* Documentação do Ket
  - https://quantumket.org
* Curso: Computação Quântica I
  - https://aprenda.quantumket.org
* Pagina do QuBOX - UFSC
  - https://qubox.ufsc.br
* Workshop de Computação Quântica - UFSC
  - https://workshop-cq.ufsc.br
* Canal do Grupo de Computação Quântica da UFSC
  - https://youtube.com/@GCQUFSC
